# Agenda, day 2

1. Recap + Q&A
2. Magic methods
3. Class attributes
4. Finding attributes with ICPO
5. Inheritance -- what it is, and how ICPO influences it
6. Three models for method inheritance
7. Data inheritance
8. What next?

# Recap

How do we solve problems? Often, it's easiest if we can use a data structure that's appropriate for the problem we're trying to solve. Objects allow us to create a data structure that is specifically designed to solve our problem, or to make it easier to think about our problem. Even if it's just a wrapper around strings, lists, tuples, dicts, etc., often the fact that an object has a different name can help us to solve our problems more easily.

The idea of object-oriented programming is: Create new data structures that are easier to use and think about than existing data structures.

- By putting these core data structures inside of an object, we can think at a higher level -- abstraction
- Methods (verbs) are defined on the class, which means that they are tightly bound to a particular type of data. Regular functions can be invoked with *any* type of value as an argument, although that might not be appropriate, and you can get an error. Methods are only available on the objects for which they actually exist. If you try to run a method on an object where it wasn't defined, you'l

